# Gamma-Ray Tutorial

This notebook demonstrates how to use TARDIS to simulate gamma-ray transport. We will walk through:

1. Setting up a TARDIS configuration for gamma-ray transport.
2. Creating a simulation state.
3. Inspecting gamma-ray decay properties.

For demonstration, we will use the same approach as in TARDIS's gamma-ray tests.

## 1. Imports

We'll need TARDIS, `astropy`, `numpy`, and `radioactivedecay` (for handling nuclear decay).

In [2]:
from pathlib import Path

import astropy.units as u

from tardis.energy_input.energy_source import get_nuclear_lines_database
from tardis.energy_input.gamma_ray_channel import (
    calculate_total_decays,
    create_inventories_dict,
    create_isotope_decay_df,
    create_isotope_dicts,
)
from tardis.io.configuration import config_reader
from tardis.model import SimulationState


/Users/wkerzend/python/tardis/tardis/__init__.py:20: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

## 2. Load Configuration and Construct Simulation State

We'll assume your TARDIS environment has an example configuration file. If you've cloned the TARDIS repo and have local data, you can adjust the path below accordingly.

In [3]:
# Modify this path as needed
example_configuration_dir = Path("./examples")  # Example placeholder
config_path = example_configuration_dir / "tardis_configv1_density_exponential_nebular_multi_isotope.yml"

config = config_reader.Configuration.from_yaml(config_path)

# For demonstration, override some parameters:
config.model.structure.velocity.start = 1.0 * u.km / u.s
config.model.structure.density.rho_0 = 5.0e2 * u.g / u.cm**3
config.supernova.time_explosion = 150 * u.d

# Create a simulation state (assuming you have an atomic dataset):
atomic_dataset = get_nuclear_lines_database()  # Placeholder
simulation_state = SimulationState.from_config(
    config,
    atom_data=atomic_dataset,
)

CRITICAL:tardis.io.configuration.config_reader:No config file named: examples/tardis_configv1_density_exponential_nebular_multi_isotope.yml


FileNotFoundError: [Errno 2] No such file or directory: 'examples/tardis_configv1_density_exponential_nebular_multi_isotope.yml'

## 3. Prepare Isotopes and Inventories

We can use utility functions from `gamma_ray_channel` to:
1. **Create an isotope dictionary** using mass fractions.
2. **Create an inventories dictionary** that can be used for decay calculations.

In [ ]:
# Get raw isotopic mass fractions & calculate cell masses
raw_isotopic_mass_fraction = simulation_state.composition.raw_isotope_abundance
cell_masses = simulation_state.composition.calculate_cell_masses(
    simulation_state.geometry.volume
)

# Create dictionary of isotopes per shell
isotope_dict = create_isotope_dicts(raw_isotopic_mass_fraction, cell_masses)

# Create inventory that can be used for decay calculations
inventories_dict = create_inventories_dict(isotope_dict)

inventories_dict

## 4. Gamma-Ray Decay Calculations

We can calculate the total decays over a given time interval (e.g., 1 day), and then construct a DataFrame describing each isotope's decay and its resulting energy channels.

In [ ]:
time_delta = 1.0 * u.d
total_decays = calculate_total_decays(inventories_dict, time_delta)
gamma_ray_lines = atomic_dataset.decay_radiation_data
isotope_decay_df = create_isotope_decay_df(total_decays, gamma_ray_lines)

isotope_decay_df.head()

## 5. Checking Energy Conservation

Below is a quick demonstration of comparing predicted total decay energy with the actual mass difference (~Q-value). We won't replicate the entire test, but the logic is similar.

In [ ]:
# Example: Ni-56 --> Co-56
nuclide_name = "Ni-56"

grouped_isotope_df = isotope_decay_df.groupby(level=["shell_number", "isotope"])
parent_isotope_energy = (
    grouped_isotope_df.get_group((0, nuclide_name.replace("-", "")))["energy_per_channel_keV"].sum()
    * u.keV
)

# Convert keV to MeV
parent_isotope_energy = parent_isotope_energy.to(u.MeV)

# Example neutrino energy from Ni-56 decay:
neutrino_energy = 0.41 * u.MeV
total_energy_actual = parent_isotope_energy + neutrino_energy

print(f"Total energy from lines + neutrino ~ {total_energy_actual:.2f}")

## Conclusion

You've now seen how to:
1. Load a TARDIS config.
2. Create a `SimulationState`.
3. Prepare isotope/inventory objects.
4. Calculate gamma-ray decays.

This is just a starting point and mirrors much of the testing logic. Explore adjusting the config, changing times, or analyzing different nuclides to see the diverse gamma-ray outputs TARDIS can generate!